# This script includes: 

## 1. Import libraries
## 2. Import data
## 3. Data pre-processing
    3.1 Reshape data
    3.2 Split data
## 4. Set up Bayesian optimizer
## 5. CNN model with optimized hyperparameters
    5.1 Run the CNN model
    5.2 Make predictions
    5.3 Classification reports
    5.4 Confusion matrices

## 1. Import libraries

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os
import seaborn as sns
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold
from keras.layers import LeakyReLU

## 2. Import data

In [2]:
# Import data

path = r'C:\Users\ashle\Documents\10-2024 Climate Analysis'

X = pd.read_csv(os.path.join(path, '02 Data Sets', 'X_weather_data_cleaned_preprocessed.csv'),index_col = False) 
y = pd.read_csv(os.path.join(path, '02 Data Sets', 'y_pleasant_cleaned_preprocessed.csv'),index_col = False) 

## 3. Data pre-processing

### 3.1 Reshape data

In [3]:
X

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.0180,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.0180,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.0180,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.0180,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.0180,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,1,0.79,1.0248,1.34,0.22,7.7,15.9,11.4,21.4,2,...,14.2,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22946,6,0.77,1.0244,1.34,0.22,5.4,16.7,14.3,21.9,0,...,14.3,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22947,4,0.76,1.0227,1.34,0.22,6.1,16.7,13.1,22.4,2,...,14.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22948,5,0.80,1.0212,1.34,0.22,5.8,15.4,11.6,21.1,1,...,12.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5


In [7]:
X.shape

(22950, 135)

In [9]:
y

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22946,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22947,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22948,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
y.shape

(22950, 15)

In [13]:
# Convert variables into numpy arrays
X = X.to_numpy()
y = y.to_numpy()

# Split X into 15 groups of 9 types of observations
X = X.reshape(-1, 15, 9)

# Check the shapes
print('X shape: ', X.shape) # should be (22950, 15, 9)
print('y shape: ', y.shape) # should be (22950, 15)

X shape:  (22950, 15, 9)
y shape:  (22950, 15)


### 3.2 Split data

In [15]:
# Split the data into two sets: a training and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 62)

In [17]:
# Check shape of each test and train set
print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (16065, 15, 9)
y_train shape:  (16065, 15)
X_test shape:  (6885, 15, 9)
y_test shape:  (6885, 15)


In [19]:
len(X_train[0])

15

In [21]:
len(X_train[0][0])

9

## 4 Set up Bayesian optimizer

In [23]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y_train)

'multilabel-indicator'

In [25]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

In [33]:
# Set up the parameters
params ={
    'activation': (0, 7),
    'neurons': (10, 100),
    'kernel': (1, 3),
    'optimizer':(0,7),
    'learning_rate': (0.0001, 0.1),
    'batch_size': (16, 128),
    'epochs':(10, 50),
    'layers1':(1,5),
    'layers2':(1,5),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.5)
}

# Activation and optimizer mappings
activation_mapping = {
    0: 'relu',
    1: 'sigmoid',
    2: 'softplus',
    3: 'softsign',
    4: 'tanh',
    5: 'selu',
    6: 'elu',
    7: 'exponential'
}
optimizer_mapping = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']

# Define Bayesian Optimization Function
def bayesian_optimization_fn(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
                             layers1, layers2, normalization, dropout, dropout_rate):
    activation_func = activation_mapping[round(activation)]
    optimizer_func = optimizer_mapping[round(optimizer)]
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(int(neurons), kernel_size=int(kernel), activation=activation_func, 
                         input_shape=(X_train.shape[1], X_train.shape[2])))
        if normalization > 0.5:
            model.add(BatchNormalization())
        for _ in range(int(layers1)):
            model.add(Dense(int(neurons), activation=activation_func))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate))
        for _ in range(int(layers2)):
            model.add(Dense(int(neurons), activation=activation_func))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(15, activation='sigmoid'))  # Multi-label classification
        model.compile(optimizer=optimizer_func, loss='binary_crossentropy') # Multi-label classification
        return model
         
    nn = KerasClassifier(build_fn=cnn_model, epochs=int(epochs), batch_size=int(batch_size), verbose=0)

    # Set up KFold
    kfold = KFold(n_splits=3, shuffle=True, random_state=42)
    
    fold_f1_scores = []

    # Manually iterate through the folds
    for fold_num, (train_idx, val_idx) in enumerate(kfold.split(X_train, y_train), 1):  # Start fold_num at 1
        # Split the data
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Fit the model
        model = cnn_model()
        history = model.fit(
            X_train_fold, 
            y_train_fold, 
            epochs=int(epochs), 
            batch_size=int(batch_size), 
            validation_data=(X_val_fold, y_val_fold),
            verbose=0
        )
        
        # Make predictions on the validation fold
        y_pred = model.predict(X_val_fold)
        y_pred = (y_pred > 0.5).astype(int)
        # Calculate the weighted F1-score
        f1 = f1_score(y_val_fold, y_pred, average='weighted')
        
        fold_f1_scores.append(f1)
        print(f'Fold {fold_num} F1 score: {f1}')

    # Calculate and return the average F1-score
    avg_f1 = np.mean(fold_f1_scores)
    print(f'Average F1 score across all folds: {avg_f1}')
    return avg_f1

# Run Bayesian Optimization
bayes_opt = BayesianOptimization(f=bayesian_optimization_fn, pbounds=params, random_state=42, verbose=2)
bayes_opt.maximize(init_points=5, n_iter=10)

# Get best parameters
best_params = bayes_opt.max['params']
print("Best parameters: ", best_params)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


C:\Users\ashle\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Fold 1 F1 score: 0.0


C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ashle\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Fold 2 F1 score: 0.0


...

C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ashle\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Fold 2 F1 score: 0.9338936252505078


C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ashle\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Fold 3 F1 score: 0.9275311586314772
Average F1 score across all folds: 0.9324420183654875
| 15        | 0.9324    | 4.117     | 39.66     | 0.8133    | 0.4051    | 49.18     | 1.327     | 3.601     | 4.743     | 0.04492   | 33.27     | 0.3678    | 0.8695    |
Best parameters:  {'activation': 3.8269719554029575, 'batch_size': 36.70369901885903, 'dropout': 0.9695846277645586, 'dropout_rate': 0.3875664116805573, 'epochs': 47.579957662567566, 'kernel': 2.789654700855298, 'layers1': 3.3915999152443406, 'layers2': 4.687496940092467, 'learning_rate': 0.008940400954986757, 'neurons': 27.638457617723066, 'normalization': 0.045227288910538066, 'optimizer': 2.2773123153428503}


C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 5. CNN model with optimized hyperparameters

### 5.1 Run the CNN model

In [35]:
# Optimized parameters
epochs = int(round(47.579957662567566))
batch_size = int(round(36.70369901885903))
n_hidden = int(round(27.638457617723066))
dropout = 0.9695846277645586
dropout_rate = 0.3875664116805573
kernel_size = int(round(2.789654700855298))
activation_value = 3.8269719554029575
optimizer_value = 2.2773123153428503
learning_rate = 0.008940400954986757
layers1 = int(round(3.3915999152443406))
layers2 = int(round(4.687496940092467))

# Activation function and optimizer mappings
activation_mapping = {
    0: 'relu',
    1: 'sigmoid',
    2: 'softplus',
    3: 'softsign',
    4: 'tanh',
    5: 'selu',
    6: 'elu',
    7: 'exponential'
}

optimizer_mapping = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']

# Get the actual activation function and optimizer from the mappings
activation = activation_mapping[round(activation_value)]  # Round the numeric value to map it
optimizer = optimizer_mapping[round(optimizer_value)]  # Round the numeric value to map it

# Input dimensions
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

# Set up the model
model = Sequential()

# Add Conv1D layer
model.add(Conv1D(n_hidden, kernel_size=kernel_size, activation=activation, input_shape=(timesteps, input_dim)))

# Add a fully connected Dense layer (layers1 neurons)
model.add(Dense(layers1, activation=activation))

# Add another Dense layer (layers2 neurons)
model.add(Dense(layers2, activation=activation))

# Add a Dropout layer
model.add(Dropout(dropout_rate))

# Add a MaxPooling layer
model.add(MaxPooling1D())

# Flatten the output for dense layers
model.add(Flatten())

# Final layer with n_classes neurons, sigmoid activation for multi-label classification
model.add(Dense(n_classes, activation='sigmoid'))

# Compile the model with the new learning rate
optimizer_instance = optimizer  # Use the optimizer obtained from mapping
model.compile(loss='binary_crossentropy', optimizer=optimizer_instance, metrics=['f1_score'])

C:\Users\ashle\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### 5.2 Make predictions

In [38]:
# Make predictions on the test set
predictions = model.predict(X_test)

# Print raw predictions (probabilities for each station)
print("Raw predictions:")
print(predictions)

# Threshold predictions for binary classification (set threshold at 0.5)
threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)

# Print the binary predictions (0s and 1s for each station)
print("Binary predictions:")
print(binary_predictions)

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step
Raw predictions:
[[0.6074804  0.5336623  0.51328754 ... 0.47866383 0.5248713  0.40688372]
 [0.5059459  0.4490594  0.40952158 ... 0.6125027  0.6241932  0.5404725 ]
 [0.54891247 0.50303775 0.7249528  ... 0.47735885 0.5337251  0.47645333]
 ...
 [0.46953437 0.44300264 0.6774686  ... 0.51548564 0.6180247  0.47458494]
 [0.6692983  0.56290746 0.37315354 ... 0.56980515 0.5373968  0.5345541 ]
 [0.59384114 0.5087777  0.4617463  ... 0.48193038 0.5266024  0.428539  ]]
Binary predictions:
[[1 1 1 ... 0 1 0]
 [1 0 0 ... 1 1 1]
 [1 1 1 ... 0 1 0]
 ...
 [0 0 1 ... 1 1 0]
 [1 1 0 ... 1 1 1]
 [1 1 0 ... 0 1 0]]


### 5.3 Classification reports

In [40]:
# Run a classsification report

# Generate predictions on training set
train_predictions = model.predict(X_train)
train_binary_predictions = (train_predictions > threshold).astype(int)

# Print classification reports for training and test sets
print("Training Set Classification Report:")
print(classification_report(y_train, train_binary_predictions))

print("Test Set Classification Report:")
print(classification_report(y_test, binary_predictions))

503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 628us/step
Training Set Classification Report:
              precision    recall  f1-score   support

           0       0.23      0.75      0.35      3984
           1       0.25      0.31      0.28      5625
           2       0.36      0.98      0.52      5237
           3       0.02      0.03      0.02      3114
           4       0.19      0.64      0.29      3480
           5       0.10      0.04      0.05      3486
           6       0.17      0.61      0.27      2696
           7       0.33      0.96      0.50      4463
           8       0.01      0.00      0.00      3370
           9       0.62      0.88      0.73      7200
          10       0.57      0.04      0.07      3381
          11       0.15      0.70      0.24      2533
          12       0.00      0.00      0.00         0
          13       0.18      0.95      0.30      2757
          14       0.06      0.45      0.10       837

   micro avg       0.23      0.57      0.33     52163
 

C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\metrics\_clas

### 5.4 Confusion matrices

In [42]:
# Define the labels (1 = pleasant, 0 = unpleasant)
labels = ['unpleasant', 'pleasant']

def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([labels[y] for y in Y_true.flatten()])
    Y_pred = pd.Series([labels[y] for y in Y_pred.flatten()])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

# Evaluate and print the confusion matrix
y_pred = (model.predict(X_test) > 0.5).astype(int)  # Apply threshold to get binary predictions
print(confusion_matrix(y_test, y_pred))

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step
Pred        pleasant  unpleasant
True                            
pleasant       12353        9500
unpleasant     42288       39134


In [46]:
#Accuracy = (TP + TN) / (TP + TN + FP + FN)

Accuracy = (12353+39134) / (12353+39134+9500+42288)
print('Accuracy: ', Accuracy)

Accuracy:  0.4985427257322682


In [48]:
# Weather stations list
weather_stations = ['BASEL', 'BELGRADE', 'BUDAPEST', 'DEBILT', 'DUSSELDORF', 'HEATHROW', 
                    'KASSEL', 'LJUBLJANA', 'MAASTRICHT', 'MADRID', 'MUNCHENB', 'OSLO', 'SONNBLICK', 
                    'STOCKHOLM', 'VALENTIA']

# Define the labels
labels = ['pleasant', 'unpleasant']

# Loop over each weather station to calculate and print the confusion matrix
for i, station in enumerate(weather_stations):
    # Get the true labels for the current station (y_test[:, i])
    true_labels = y_test[:, i]  # Using the index of the weather station in y_test
    
    # Get the predicted labels for the current station (binary_predictions[:, i])
    predicted_labels = binary_predictions[:, i]
    
    # Create the confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)
    
    # Convert confusion matrix to DataFrame for better readability
    cm_df = pd.DataFrame(cm, columns=labels, index=labels)
    
    # Print the confusion matrix for this station
    print(f"Confusion Matrix for {station}:")
    print(cm_df)
    print("\n" + "-"*50 + "\n")

Confusion Matrix for BASEL:
            pleasant  unpleasant
pleasant         751        4454
unpleasant       426        1254

--------------------------------------------------

Confusion Matrix for BELGRADE:
            pleasant  unpleasant
pleasant        2321        2197
unpleasant      1615         752

--------------------------------------------------

Confusion Matrix for BUDAPEST:
            pleasant  unpleasant
pleasant         647        4044
unpleasant        47        2147

--------------------------------------------------

Confusion Matrix for DEBILT:
            pleasant  unpleasant
pleasant        3586        1954
unpleasant      1298          47

--------------------------------------------------

Confusion Matrix for DUSSELDORF:
            pleasant  unpleasant
pleasant        1231        4200
unpleasant       511         943

--------------------------------------------------

Confusion Matrix for HEATHROW:
            pleasant  unpleasant
pleasant        4894    